Base de datos de COVID, actualizado al 01 de julio de 2020. 

In [1]:
import pandas as pd

In [2]:
# Read main dataset
data = pd.read_csv("200701COVID19MEXICO.csv", encoding='latin1')

In [3]:
data.head()

FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
0          2020-07-01      1933bd       2       3          27     2   
1          2020-07-01      1df826       2       4           9     2   
2          2020-07-01      192d67       2       4           9     2   
3          2020-07-01      054c36       2       4          25     2   
4          2020-07-01      1537b4       2       4          15     2   

   ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ... CARDIOVASCULAR  \
0           27           27              4              1  ...              2   
1           15            9             13              2  ...              2   
2           15           15              2              1  ...              2   
3           25           25              6              1  ...              2   
4           15           15             31              2  ...              2   

  OBESIDAD RENAL_CRONICA  TABAQUISMO  OTRO_CASO  RESULTADO  MIGRANTE  \
0        2             2           2          1          1        99   
1        2             1           2         99          1        99   
2        1             2           2         99          1        99   
3        2             2           2         99          1        99   
4        2             2           2         99          1        99   

   PAIS_NACIONALIDAD  PAIS_ORIGEN  UCI  
0            MÃ©xico           99   97  
1            MÃ©xico           99    2  
2            MÃ©xico           99   97  
3            MÃ©xico           99   97  
4            MÃ©xico           99    2  

[5 rows x 35 columns]

## Create inegi code

In [4]:
data['ENTIDAD_RES'].nunique()

32

In [5]:
#Check identifier
ids = data['ID_REGISTRO'].unique()
len(ids) == len(data)
len(ids)

595917

In [6]:
ent = data['ENTIDAD_RES'].astype(str)
mun = data['MUNICIPIO_RES'].astype(str)

In [7]:
# Create column with inegi codes -> ent + mun
inegi_mun = []
for i in mun:
    new_num = ""
    if len(i)==1:
        new_num = "00"+i
    elif len(i)==2:
        new_num = "0"+i
    else:
        new_num = i
    inegi_mun.append(new_num)

inegi_code = ent+inegi_mun
data["clave_inegi"] = inegi_code

## Import OID codes

In [8]:
# Import codes of municipalities
codes = pd.read_csv("codes_municipality.csv")

# Change numeric to object
codes["OID"] = codes["OID"].astype(str)
codes["code of municipality"] = codes["code of municipality"].astype(str)
codes.dtypes

OID                     object
State                   object
code of municipality    object
Municipality            object
dtype: object

In [9]:
# Clean OID column
oid_raw = codes["OID"]
oid_list = []
for i in oid_raw:
    new = i.split(".")
    oid_list.append(new[0])
#oid_list

codes["OID"] = oid_list
codes.head()

OID        State code of municipality            Municipality
0   1  Mexico City                 9012                 Tlalpan
1   2  Mexico City                 9013              Xochimilco
2   3  Mexico City                 9008  La Magdalena Contreras
3   4  Mexico City                 9002            Azcapotzalco
4   5  Mexico City                 9014           Benito Juarez

In [10]:
codes['code of municipality'].isna().sum()

0

In [11]:
# Create tuples (oid, inegi code of municipality)
#Do not change !!!
tuples = codes[['OID', 'code of municipality']].apply(tuple, axis=1)
tuples

0           (1, 9012)
1           (2, 9013)
2           (3, 9008)
3           (4, 9002)
4           (5, 9014)
            ...      
2452    (2453, 23002)
2453    (2454, 23001)
2454    (2455, 23007)
2455    (2456, 23003)
2456       (0, 23010)
Length: 2457, dtype: object

In [12]:
tuples2 = codes[['State','code of municipality']].apply(tuple, axis=1)
tuples2[0]

('Mexico City', '9012')

In [13]:
tuples3 = codes[['Municipality','code of municipality']].apply(tuple, axis=1)
tuples3

0                       (Tlalpan, 9012)
1                    (Xochimilco, 9013)
2        (La Magdalena Contreras, 9008)
3                  (Azcapotzalco, 9002)
4                 (Benito Juarez, 9014)
                     ...               
2452    (Felipe Carrillo Puerto, 23002)
2453                   (Cozumel, 23001)
2454           (Lazaro Cardenas, 23007)
2455              (Isla Mujeres, 23003)
2456                   (Bacalar, 23010)
Length: 2457, dtype: object

In [14]:
# function to read inegi code and change to oid
def set_value(value, tuples=tuples):
    #oid_values = []
    for t in tuples:
        if value == t[1]:
            new_value = t[0]
            return new_value

In [15]:
# Apply function to each row
data["OID"] = data["clave_inegi"].apply(lambda x: set_value(x))

In [16]:
data[data['clave_inegi']=='32058']

FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
300257          2020-07-01      17beeb       1      99          32     1   
309013          2020-07-01      1ba218       1      99          32     2   

        ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
300257           10           32             58              2  ...   
309013           32           32             58              2  ...   

       RENAL_CRONICA TABAQUISMO OTRO_CASO  RESULTADO  MIGRANTE  \
300257             2          2        99          2        99   
309013             2          2        99          2        99   

        PAIS_NACIONALIDAD  PAIS_ORIGEN  UCI  clave_inegi  OID  
300257            MÃ©xico           99    2        32058  776  
309013            MÃ©xico           99    2        32058  776  

[2 rows x 37 columns]

In [17]:
# Apply function to each row
data["state"] = data["clave_inegi"].apply(lambda x: set_value(x, tuples=tuples2))

In [18]:
# Apply function to each row
data["municipality"] = data["clave_inegi"].apply(lambda x: set_value(x, tuples=tuples3))

In [19]:
missing_OID = data[data['OID'].isna()]
missing_OID.to_excel("missing_oid_codes.xlsx")

In [20]:
#Drop unnecesary columns
data = data.drop(["ENTIDAD_UM","ENTIDAD_RES","ENTIDAD_NAC","MUNICIPIO_RES",
                  "TIPO_PACIENTE","PAIS_NACIONALIDAD","PAIS_ORIGEN","SECTOR",
                 "ORIGEN","FECHA_ACTUALIZACION"], axis=1)

In [21]:
# Change name of selected columns
data = data.rename(columns={"ID_REGISTRO":"id",
                           "SEXO": "sex",
                           "FECHA_INGRESO":"date",
                           "FECHA_SINTOMAS":"date_symptoms",
                           "FECHA_DEF":"date_death",
                           "RESULTADO":"covid_test",
                           "clave_inegi":"inegi_code",
                           "EDAD": "age",
                           "HABLA_LENGUA_INDIG":"indigenous"})

In [22]:
data.head()

id  sex        date date_symptoms  date_death  INTUBADO  NEUMONIA  age  \
0  1933bd    2  2020-04-22    2020-04-22  9999-99-99        97         2   39   
1  1df826    2  2020-03-21    2020-03-16  2020-03-22         2         1   78   
2  192d67    2  2020-04-22    2020-04-12  9999-99-99        97         2   34   
3  054c36    2  2020-04-24    2020-04-21  9999-99-99        97         2   41   
4  1537b4    2  2020-04-27    2020-04-22  2020-04-28         2         1   75   

   NACIONALIDAD  EMBARAZO  ...  RENAL_CRONICA  TABAQUISMO  OTRO_CASO  \
0             1        97  ...              2           2          1   
1             1        97  ...              1           2         99   
2             1        97  ...              2           2         99   
3             1        97  ...              2           2         99   
4             1        97  ...              2           2         99   

   covid_test  MIGRANTE  UCI  inegi_code  OID            state  municipality  
0           1        99   97       27004  906          Tabasco        Centro  
1           1        99    2        9013    2      Mexico City    Xochimilco  
2           1        99   97       15002  221  State of Mexico       Acolman  
3           1        99   97       25006  258          Sinaloa      Culiacan  
4           1        99    2       15031  193  State of Mexico  Chimalhuacan  

[5 rows x 29 columns]

In [23]:
#Convert sex to categorical
sex = data["sex"]
sex_cat = []
for s in sex:
    if s==1:
        cat = "woman"
    elif s==2:
        cat = "man"
    else:
        cat = "NA"
    sex_cat.append(cat)
len(sex_cat)
data["sex_cat"] = sex_cat

In [24]:
# Convert death to categorical
death = data["date_death"]
death_new_value = []
for d in death:
    if d == "9999-99-99":
        value = "NA"
    else: 
        value = d
    death_new_value.append(value)
len(death_new_value)
data["date_death"] = death_new_value

In [25]:
# Create counter of positive cases
covid_num = data["covid_test"]
num = []
for c in covid_num:
    if c==1:
        n=1
    else:
        n=0
    num.append(n)
data["covid_confirmed"] = num

In [26]:
# convert ctest to categorical
ctest = data["covid_confirmed"]
ctest_cat = []
for c in ctest:
    if c==1:
        cat = "positive"
    elif c==0:
        cat = "negative"
    else:
        cat = "NA"
    ctest_cat.append(cat)
len(ctest_cat)
data["covid_test_cat"] = ctest_cat

In [27]:
# convert indigenous to categorical
ind = data["indigenous"]
ind_cat = []
for i in ind:
    if i==1:
        cat="yes"
    elif i==2:
        cat="no"
    else:
        cat="NA"
    ind_cat.append(cat)
len(ind_cat)
data["indigenous_cat"] = ind_cat

In [28]:
# Create dead column
confirmed_death = data['date_death'].fillna(0).replace("NA",0)

total = []
for i in confirmed_death:
    if i == 0:
        count = 0
    else:
        count = 1
    total.append(count)
total
data["dead_confirmed"] = total

# Total dead with covid confirmed
#with_covid = df[df['covid_confirmed']==1]
#with_covid["dead_confirmed"].sum()

In [29]:
data.head()

id  sex        date date_symptoms  date_death  INTUBADO  NEUMONIA  age  \
0  1933bd    2  2020-04-22    2020-04-22          NA        97         2   39   
1  1df826    2  2020-03-21    2020-03-16  2020-03-22         2         1   78   
2  192d67    2  2020-04-22    2020-04-12          NA        97         2   34   
3  054c36    2  2020-04-24    2020-04-21          NA        97         2   41   
4  1537b4    2  2020-04-27    2020-04-22  2020-04-28         2         1   75   

   NACIONALIDAD  EMBARAZO  ...  UCI  inegi_code  OID            state  \
0             1        97  ...   97       27004  906          Tabasco   
1             1        97  ...    2        9013    2      Mexico City   
2             1        97  ...   97       15002  221  State of Mexico   
3             1        97  ...   97       25006  258          Sinaloa   
4             1        97  ...    2       15031  193  State of Mexico   

   municipality  sex_cat  covid_confirmed  covid_test_cat  indigenous_cat  \
0        Centro      man                1        positive              NA   
1    Xochimilco      man                1        positive              no   
2       Acolman      man                1        positive              no   
3      Culiacan      man                1        positive              no   
4  Chimalhuacan      man                1        positive              no   

   dead_confirmed  
0               0  
1               1  
2               0  
3               0  
4               1  

[5 rows x 34 columns]

In [30]:
# Total with covid
data["dead_confirmed"].unique()

array([0, 1])

In [31]:
# convert to date and extract month and day
data['date'] = pd.to_datetime(data['date'])
data['month'] = pd.DatetimeIndex(data['date']).month
data['day'] = pd.DatetimeIndex(data['date']).day
data['week'] = pd.DatetimeIndex(data['date']).week # Convert to week

In [32]:
# Create column with month name
month_num = data['month']
names = []
for num in month_num:
    if num == 1:
        name = "Jan"
    elif num ==2:
        name = "Feb"
    elif num ==3:
        name = "Mar"
    elif num ==4:
        name = "Apr"
    elif num ==5:
        name = "May"
    elif num ==6:
        name = "Jun"
    else:
        name = "NaN"
    names.append(name)
data['month_name'] = names
data.head()  

id  sex       date date_symptoms  date_death  INTUBADO  NEUMONIA  age  \
0  1933bd    2 2020-04-22    2020-04-22          NA        97         2   39   
1  1df826    2 2020-03-21    2020-03-16  2020-03-22         2         1   78   
2  192d67    2 2020-04-22    2020-04-12          NA        97         2   34   
3  054c36    2 2020-04-24    2020-04-21          NA        97         2   41   
4  1537b4    2 2020-04-27    2020-04-22  2020-04-28         2         1   75   

   NACIONALIDAD  EMBARAZO  ...  municipality  sex_cat  covid_confirmed  \
0             1        97  ...        Centro      man                1   
1             1        97  ...    Xochimilco      man                1   
2             1        97  ...       Acolman      man                1   
3             1        97  ...      Culiacan      man                1   
4             1        97  ...  Chimalhuacan      man                1   

   covid_test_cat  indigenous_cat  dead_confirmed  month  day  week  \
0        positive              NA               0      4   22    17   
1        positive              no               1      3   21    12   
2        positive              no               0      4   22    17   
3        positive              no               0      4   24    17   
4        positive              no               1      4   27    18   

   month_name  
0         Apr  
1         Mar  
2         Apr  
3         Apr  
4         Apr  

[5 rows x 38 columns]

In [33]:
#Create column with age category
age_num = data['age'].astype(int)
ages_cat = []
for num in age_num:
    if num < 50:
        if num >=25:
            name = "25-49 years"
        else:
            name = "0-24 years" 
    elif num >= 50:
        if num >= 65:
            name = "65 years or more"
        else:
            name = "50-64 years"            
    else:
        name = "Not specified"
    ages_cat.append(name)
data['ages_cat'] = ages_cat
data.head()

id  sex       date date_symptoms  date_death  INTUBADO  NEUMONIA  age  \
0  1933bd    2 2020-04-22    2020-04-22          NA        97         2   39   
1  1df826    2 2020-03-21    2020-03-16  2020-03-22         2         1   78   
2  192d67    2 2020-04-22    2020-04-12          NA        97         2   34   
3  054c36    2 2020-04-24    2020-04-21          NA        97         2   41   
4  1537b4    2 2020-04-27    2020-04-22  2020-04-28         2         1   75   

   NACIONALIDAD  EMBARAZO  ...  sex_cat  covid_confirmed  covid_test_cat  \
0             1        97  ...      man                1        positive   
1             1        97  ...      man                1        positive   
2             1        97  ...      man                1        positive   
3             1        97  ...      man                1        positive   
4             1        97  ...      man                1        positive   

   indigenous_cat  dead_confirmed  month  day  week  month_name  \
0              NA               0      4   22    17         Apr   
1              no               1      3   21    12         Mar   
2              no               0      4   22    17         Apr   
3              no               0      4   24    17         Apr   
4              no               1      4   27    18         Apr   

           ages_cat  
0       25-49 years  
1  65 years or more  
2       25-49 years  
3       25-49 years  
4  65 years or more  

[5 rows x 39 columns]

In [34]:
ind = data['indigenous_cat'].fillna(0).replace("NA",0)
total = []
for i in ind:
    if i == "no" or i == 0:
        count = 0
    elif i =="yes":
        count = 1
    total.append(count)
total
data["indigenous_count"] = total
data["indigenous_count"].unique()

array([0, 1])

In [35]:
data.to_csv("main_dataset_covid.csv")

## Create small dataset

In [36]:
# Create small dataframe
df = data.loc[:,["OID","inegi_code","state","municipality","id","covid_confirmed","covid_test","covid_test_cat",
                "date","date_symptoms","date_death","sex_cat","age","indigenous_cat"]]
df.head()

OID inegi_code            state  municipality      id  covid_confirmed  \
0  906      27004          Tabasco        Centro  1933bd                1   
1    2       9013      Mexico City    Xochimilco  1df826                1   
2  221      15002  State of Mexico       Acolman  192d67                1   
3  258      25006          Sinaloa      Culiacan  054c36                1   
4  193      15031  State of Mexico  Chimalhuacan  1537b4                1   

   covid_test covid_test_cat       date date_symptoms  date_death sex_cat  \
0           1       positive 2020-04-22    2020-04-22          NA     man   
1           1       positive 2020-03-21    2020-03-16  2020-03-22     man   
2           1       positive 2020-04-22    2020-04-12          NA     man   
3           1       positive 2020-04-24    2020-04-21          NA     man   
4           1       positive 2020-04-27    2020-04-22  2020-04-28     man   

   age indigenous_cat  
0   39             NA  
1   78             no  
2   34             no  
3   41             no  
4   75             no

In [36]:
#Check null values
len(data) - df["OID"].isnull().sum()

595807

In [37]:
df = df.dropna()
len(df)

595807

In [39]:
df.dtypes

OID                object
inegi_code         object
state              object
municipality       object
id                 object
covid_confirmed     int64
covid_test          int64
covid_test_cat     object
date               object
date_symptoms      object
date_death         object
sex_cat            object
age                 int64
indigenous_cat     object
dtype: object

In [62]:
df.to_csv("covid_data_01jul.csv", index=False)

In [ ]:
#Convert to csv and txt
#df.to_csv("covid_data_26june.text", index = False)
#df.to_csv("covid_data_28june.csv", index = False)